## AMPL - Introduction to music21 (2)
#### This notebook presents some new methods of music21, as well as specific characteristics of the jingju and makam scores.

In [1]:
from music21 import *

### Working with jingju scores

We are going to work with the vocal part of a jingju score. So let's save it in a variable `p`.

In [2]:
s = converter.parse('lseh-YiLunMing-WenZhaoGuan-1.xml')
pp = s.parts # all the parts from the score
p = pp[0] # in this score, the vocal part is the first one
p.show()

To work with the music, we should focus on the notes, which are contained in the measures. So let's retrive all measures and notes and save them respectively in the variables `mm` and `nn` using the `.getElementsByClass` method.
<br/>
As always, everything retrieved should be saved as a stream.

In [3]:
mm = p.getElementsByClass(stream.Measure).stream() # all the measures from the part
nn = p.getElementsByClass(note.Note).stream() # all the notes from the part
print('Measures in the part:', len(mm))
print('Notes in the part:', len(nn))

Measures in the part: 437
Notes in the part: 0


As you can see, no notes were retrieved. This is due to the fact that notes are not contained in the part (`p`), but in each of its measures. So, in order to retrieve all the notes, we can apply the `.getElementsByClass` method in a for loop.

In [4]:
nn = stream.Stream() # create a stream to store all the notes
for m in mm:
    for n in m.getElementsByClass(note.Note).stream():
        nn.append(n)
print('Notes in the part:', len(nn))

Notes in the part: 1175


Since working with notes is the goal of music21, it conveniently offers the `.notes` attribute precisely for that.

In [5]:
nn = stream.Stream()
for m in mm:
    for n in m.notes.stream(): # NOTE: .notes
        nn.append(n)
print('Notes in the part', len(nn))

Notes in the part 1175


As with the case of `.getElementsByClass`, the `.notes` attribute only retrives notes from the stream to which it is applied.

In [6]:
nn = p.notes.stream()
print('Notes in the part', len(nn))

Notes in the part 0


However, music21 offers the `.flat` attribute allows for searching in the streams contained in the stream to which it is applied.

In [7]:
nn = p.flat.notes.stream()  # NOTE: .flat
print('Notes in the part', len(nn))

Notes in the part 1175


Now we have all the notes (and only the notes, no rests) stored in the stream `nn` that we created. Since we only stored notes there, there is no information about key nor time signature.

In [8]:
nn.show()

Interestingly, using the `.flat` attribute, the notes now relate their offset to the stream `nn`. Therefore, their offset is given with respect to the whole score.

In [9]:
print('The first note of this part is in offset', nn[0].offset)

The first note of this part is in offset 36.0


Jingju arias usually have *banshi* changes, which are indicated in the score by a change of time signature. Let's see if this is the case.

In [10]:
tSs = p.flat.getElementsByClass(meter.TimeSignature).stream() # NOTE: .flat is required
print('This score contains {} time signatures'.format(len(tSs)))

This score contains 2 time signatures


Let's look at these time signatures

In [11]:
for tS in tSs:
    print(str(tS.numerator)+'/'+str(tS.denominator), 'in measure', tS.measureNumber, 'at offset', tS.offset)

4/4 in measure 1 at offset 0.0
2/4 in measure 135 at offset 536.0


#### Working with intervals

Intervals are a special type of object in music21 that has to be created from two notes. Let's explore the interval between the first 2 notes of this part.

In [12]:
n0 = nn[0]
n1 = nn[1]
itv1 = interval.Interval(n0, n1)
print(itv1)
print('Interval from {} to {}'.format(itv1.noteStart.name, itv1.noteEnd.name))
print('Name of the interval:', itv1.name)
print('Simple name:', itv1.simpleName)
print('Nicer name:', itv1.niceName)
print('Interval semitones:', itv1.semitones)
print('Interval cents:', itv1.cents)

<music21.interval.Interval m3>
Interval from F# to A
Name of the interval: m3
Simple name: m3
Nicer name: Minor Third
Interval semitones: 3
Interval cents: 300.0


Sometimes we need to know the direction of the interval. Let's compare the previous interval with the one formed by the next to notes in the part.

In [13]:
n2 = nn[2]
itv2 = interval.Interval(n1, n2)
print('Direction of the interval from {} to {}: {}'.format(itv1.noteStart.nameWithOctave, itv1.noteEnd.nameWithOctave, itv1.direction.name))
print('Direction of the interval from {} to {}: {}'.format(itv2.noteStart.nameWithOctave, itv2.noteEnd.nameWithOctave, itv2.direction.name))

Direction of the interval from F#4 to A4: ASCENDING
Direction of the interval from A4 to G#4: DESCENDING


There are different ways to work with interval direction.

In [14]:
print('Name of the interval:\t', itv2.name, '\t\t', itv2.directedName)
print('Simple name:\t\t', itv2.simpleName, '\t\t', itv2.directedSimpleName)
print('Nicer name:\t\t', itv2.niceName, '\t', itv2.directedNiceName)
print('Interval semitones:', itv2.semitones)
print('Interval cents:', itv2.cents)

Name of the interval:	 m2 		 m-2
Simple name:		 m2 		 m-2
Nicer name:		 Minor Second 	 Descending Minor Second
Interval semitones: -1
Interval cents: -100.0


#### Extra functionalities with notes

If we need to work with pairs of consequtive notes (for example, to analyse the intervals in a piece), music21 conveniently offers the methos `.previous()` and `.next()` to retrive respectively the previous and following notes.

In [15]:
print('{} is preceded by {} and followed by {}'.format(n1.name, n1.previous().name, n1.next().name))

A is preceded by F# and followed by G#


We can change the color of a note, in case, for example, that we need to find particular ones in the score.

To test this functionality, let's search for all the perfect fourths in the score, and color the first note in red, and the second in purple.

In [16]:
p4count = 0 # let's also count how many intervals are present in the part.

for n in nn[:-1]:
    itv = interval.Interval(n, n.next())
    if itv.name == 'P4':
        itv.noteStart.style.color = 'red'   # NOTE: .style.color
        itv.noteEnd.style.color = 'purple'  # NOTE: .style.color
        p4count += 1

print('This part conatins {} perfect fourths'.format(p4count))

nn.show()

This part conatins 64 perfect fourths


When we retrieve an object, we create a reference to the original one, not a copy of it. Therefore, the changes we do to it affect the original object.

In [17]:
s.show()

If you pay attention to the results, you'll see that some intervals where found in which there is a rest between the two notes of the interval. This might be not perceived as an interval by the listener.
<br/>
This is due to fact that when we call the `.notes` attribute, it only retrieves notes, and only notes, not rests. If we need to consider rests, we should use the `.notesAndRests` method.

In [18]:
nr = p.flat.notesAndRests.stream()
#nr.show()
print('This part contains {} notes'.format(len(nn)))
print('This part contains {} notes and rests'.format(len(nr)))

This part contains 1175 notes
This part contains 1510 notes and rests


To try this new approach, we will need to load the score again. Otherwise, all the notes whose color was changed previously, will continue with their new color.

In [18]:
s = converter.parse('lseh-YiLunMing-WenZhaoGuan-1.xml')
p = s.parts[0]
nr = p.flat.notesAndRests.stream()

p4count = 0
for n in nr[:-1]:
    if n.isNote and n.next().isNote:      # NOTE: .isNote
        itv = interval.Interval(n, n.next())
        if itv.name == 'P4':
            itv.noteStart.style.color = 'red'
            itv.noteEnd.style.color = 'purple'
            p4count += 1

print('This part contains {} perfect fourths'.format(p4count))

nr.show()

This part contains 56 perfect fourths


Jingu scores have two parts, one vocal and one instrumental. Few of them even have three parts, two vocal parts, and one instrumental part. However, different sources have the vocal part on top, while others have it as the second part. If you work with a collection of scores, you might need to know which part is the vocal one and which one is the instrumental one. A way to automatically identify this is by searching for the part with lyrics.
<br/>
This is my way of doing it:

In [26]:
s = converter.parse('lseh-YiLunMing-WenZhaoGuan-1.xml')

vocalParts = []
instrumentalPart = []

for p in s.parts:
    nn = p.flat.notes.stream() # to look for lyrics, only notes are needed
    i = 0
    n = nn[i]
    print(nn.id)
    # sometimes the vocal part starts with grace notes,
    # but lyrics are only attached to main notes
    while n.quarterLength == 0:
        i += 1
        n = nn[i]
    if n.lyric != None:
        vocalParts.append(p)
    else:
        instrumentalPart.append(p)

print('There is {} vocal part and {} instrumental part'.format(len(vocalParts), len(instrumentalPart)))

#vocalParts[0].show()

Piano_flat
Piano_flat
There is 1 vocal part and 1 instrumental part


In [25]:
nn.__dict__

{'_activeSite': None,
 '_naiveOffset': 0.0,
 '_activeSiteStoredOffset': None,
 '_derivation': <Derivation of <music21.stream.Part Piano_flat> from <music21.stream.Part Piano_flat> via "notes">,
 '_style': None,
 '_editorial': None,
 '_duration': None,
 '_priority': 0,
 'id': 'Piano_flat',
 'groups': ['P2'],
 'sites': <music21.sites.Sites at 0x125446688>,
 '_cache': {},
 '_offsetDict': {4985648800: (2.5, <music21.note.Note A>),
  4985649080: (2.75, <music21.note.Note B>),
  4985649360: (3.0, <music21.note.Note G#>),
  4985649640: (3.5, <music21.note.Note F#>),
  4985649920: (3.75, <music21.note.Note G#>),
  5000440912: (4.0, <music21.note.Note E>),
  5000441192: (4.5, <music21.note.Note C#>),
  5000441472: (5.0, <music21.note.Note B>),
  5000441752: (5.5, <music21.note.Note G#>),
  5000245488: (6.0, <music21.note.Note F#>),
  5000245768: (6.5, <music21.note.Note G#>),
  5000246048: (7.0, <music21.note.Note G#>),
  5000246328: (7.5, <music21.note.Note C#>),
  5000246664: (8.0, <music21.n

### Working with makam scores
Most of makam scores have non standard key signatures, and this might cause problems.

In [21]:
makamScore = 'segah--sazsemaisi--aksaksemai----hizir_aga.xml'
s = converter.parse(makamScore)

MusicXMLImportException: For non traditional signatures each step must have an alter

Music21 was not developed with makam music in mind, and many of the accidentals used in makam music are not recognized by music21.

In [22]:
# these are the names of all the accidentals used in makam scores, as contained in the MusicXML files
makamAccidentals = ['double-slash-flat', 'flat', 'slash-flat', 'quarter-flat', 'quarter-sharp', 'sharp', 'slash-quarter-sharp', 'slash-sharp']

# create a stream to contained altered notes
makamNotes = stream.Stream()

for i in range(len(makamAccidentals)): # create a note per accidental
    try:
        n = note.Note()
        n.pitch.accidental = makamAccidentals[i] # add one accidental from the list
        n.addLyric(makamAccidentals[i], applyRaw=True) # add the name of the accidental as lyric
        n.addLyric(n.pitch.accidental.name, applyRaw=True) # add the name used by music21 as lyric
        n.addLyric(n.pitch.accidental.alter) # add the number of semitones of the accidental as lyric
        makamNotes.append(n)
    except:
        print("music21 doesn't accept {} as accidental".format(makamAccidentals[i]))

makamNotes.show()

music21 doesn't accept double-slash-flat as accidental
music21 doesn't accept slash-flat as accidental
music21 doesn't accept slash-quarter-sharp as accidental
music21 doesn't accept slash-sharp as accidental


Since the problem when loading the score with music21 is the non standard key signature, one solution might be to manipulate the MusicXML file to get rid of the time signature.
<br/>
I use ElementTree to find it.

In [23]:
import xml.etree.ElementTree as ET

tree = ET.parse(makamScore)
root = tree.getroot()        

notes = []
accidentals = []

for k in root.iter('key'):
    for ks in k.findall('key-step'):
        notes.append(ks.text)
    for ka in k.findall('key-accidental'):
        accidentals.append(ka.text)

print('The key signature of this score has:')
for i in range(len(notes)):
    print('-', notes[i], accidentals[i])

The key signature of this score has:
- F sharp
- E quarter-flat
- B quarter-flat


Now I can remove it from the MusicXML file and create a new file without key signature

In [24]:
for k in root.iter('key'):
    for ks in k.findall('key-step'):
        k.remove(ks)
    for ka in k.findall('key-accidental'):
        k.remove(ka)

newMakamScore = makamScore[:-4] + '_withoutKeySignature.xml'

tree.write(newMakamScore)

And now, music21 will load the score

In [25]:
s = converter.parse(newMakamScore)
s.show()

Even without the key signature, the information about accidentals is stored in the MusicXML file within each note, so it is still available

In [27]:
nn = s.parts[0].flat.notes.stream()
for n in nn:
    if n.pitch.accidental != None:
        print(n.pitch.accidental.name)

half-flat
half-flat
half-flat
sharp
sharp
half-flat
sharp
half-flat
sharp
half-flat
half-flat
sharp
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
sharp
sharp
half-flat
sharp
half-flat
sharp
half-flat
half-flat
sharp
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
sharp
sharp
half-flat
sharp
half-flat
sharp
half-flat
sharp
sharp
sharp
sharp
half-flat
half-flat
half-flat
half-flat
sharp
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
sharp
sharp
sharp
sharp
half-flat
sharp
sharp
sharp
sharp
half-flat
half-flat
half-flat
half-flat
sharp
half-flat
half-flat
half-flat
half-flat
half-flat
half-flat
sharp
sharp
half-flat
sharp
half-flat
sharp
half-flat
sharp
half-flat
sharp
half-flat
sharp
half-flat
half-flat
sharp
half-flat
sharp
half-flat
sharp
half-flat
sharp
sharp
half-flat
half-flat
half-flat
sharp
sharp
sharp
half-flat
sharp
half-flat
sharp
half-flat
sharp
half